<a href="https://colab.research.google.com/github/cojocarucosmin/AICourseDev/blob/main/CSV_and_Database_Assistant_with_LangChain_Agents_and_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install required libraries

In [1]:
!pip install -q openai langchain-openai langchain-community langchain-core langchain_experimental gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0

## Create setup

In [2]:
import os, openai
import pandas as pd
import gradio as gr

from IPython.display import Markdown, HTML, display
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage

from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [3]:
# only for Google Colab; please comment Kaggle part in this case
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# only for Kaggle; please comment Google Colab part in this case
#from kaggle_secrets import UserSecretsClient
#user_secrets = UserSecretsClient()
#os.environ["OPENAI_API_KEY"] = user_secrets.get_secret("OPENAI_API_KEY")

## Get data from internet and save it locally

In [4]:
os.makedirs("data",exist_ok=True)
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)

--2024-09-26 10:19:38--  https://covidtracking.com/data/download/all-states-history.csv
Resolving covidtracking.com (covidtracking.com)... 172.67.183.132, 104.21.64.114, 2606:4700:3032::ac43:b784, ...
Connecting to covidtracking.com (covidtracking.com)|172.67.183.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘./data/all-states-history.csv’

all-states-history.     [ <=>                ]   2.61M  --.-KB/s    in 0.1s    

2024-09-26 10:19:39 (22.3 MB/s) - ‘./data/all-states-history.csv’ saved [2738601]



## Explore the data

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20780 entries, 0 to 20779
Data columns (total 41 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   date                              20780 non-null  object 
 1   state                             20780 non-null  object 
 2   death                             20780 non-null  float64
 3   deathConfirmed                    20780 non-null  float64
 4   deathIncrease                     20780 non-null  int64  
 5   deathProbable                     20780 non-null  float64
 6   hospitalized                      20780 non-null  float64
 7   hospitalizedCumulative            20780 non-null  float64
 8   hospitalizedCurrently             20780 non-null  float64
 9   hospitalizedIncrease              20780 non-null  int64  
 10  inIcuCumulative                   20780 non-null  float64
 11  inIcuCurrently                    20780 non-null  float64
 12  nega

In [6]:
df

,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2021-03-07,AK,305.0,0.0,0,0.0,1293.0,1293.0,33.0,0,...,1731628.0,0,0.0,0.0,0.0,0.0,0.0,0,1731628.0,0
1,2021-03-07,AL,10148.0,7963.0,-1,2185.0,45976.0,45976.0,494.0,0,...,2323788.0,2347,0.0,0.0,119757.0,0.0,2323788.0,2347,0.0,0
2,2021-03-07,AR,5319.0,4308.0,22,1011.0,14926.0,14926.0,335.0,11,...,2736442.0,3380,0.0,0.0,0.0,481311.0,0.0,0,2736442.0,3380
3,2021-03-07,AS,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,2140.0,0,0.0,0.0,0.0,0.0,0.0,0,2140.0,0
4,2021-03-07,AZ,16328.0,14403.0,5,1925.0,57907.0,57907.0,963.0,44,...,7908105.0,45110,580569.0,0.0,444089.0,0.0,3842945.0,14856,7908105.0,45110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20775,2020-01-17,WA,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
20776,2020-01-16,WA,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
20777,2020-01-15,WA,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
20778,2020-01-14,WA,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0


## Prepare the Langchain dataframe agent

In [24]:
# Create instance of ChatOpenAI with the specified model and temperature setting
model = ChatOpenAI(temperature=0.5, model='gpt-4')

# Create agent to query the csv data
agent = create_pandas_dataframe_agent(llm=model,
                                      df=df,
                                      verbose=True,
                                      allow_dangerous_code=True)

## Start asking questions

In [25]:
question = "how many rows are there for US?"

agent.invoke(question)



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to filter rows where 'state' column is 'US' and then count the number of these rows.
Action: python_repl_ast
Action Input: len(df[df['state'] == 'US'])0I now know the final answer.
Final Answer: There are no rows for 'US' in the 'state' column.

> Finished chain.


{'input': 'how many rows are there for US?',
 'output': "There are no rows for 'US' in the 'state' column."}

## Build template prompt

In [26]:
prompt_prefix = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

prompt_suffix = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

In [27]:
question = "How may patients were hospitalized in July 2020 in Texas"

agent.invoke(prompt_prefix + question + prompt_suffix)



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to select rows where the `state` is 'TX' and the `month_year` is '2020-07', then sum the `hospitalized` column. However, first I need to set the pandas display options to show all the columns and get the column names to ensure that I'm using the correct column names.
Action: python_repl_ast
Action Input: 
```python
import pandas as pd

# Set pandas display options
pd.set_option('display.max_columns', None)

# Print column names
print(df.columns)
```Index(['date', 'state', 'death', 'deathConfirmed', 'deathIncrease',
       'deathProbable', 'hospitalized', 'hospitalizedCumulative',
       'hospitalizedCurrently', 'hospitalizedIncrease', 'inIcuCumulative',
       'inIcuCurrently', 'negative', 'negativeIncrease',
       'negativeTestsAntibody', 'negativeTestsPeopleAntibody',
       'negativeTestsViral', 'onVentilatorCumulative', 'onVentilatorCurrently',
       'positive', 'positiveCasesViral', 'positiveIncreas

{'input': '\nFirst set the pandas display options to show all the columns,\nget the column names, then answer the question.\nHow may patients were hospitalized in July 2020 in Texas\n- **ALWAYS** before giving the Final Answer, try another method.\nThen reflect on the answers of the two methods you did and ask yourself\nif it answers correctly the original question.\nIf you are not sure, try another method.\n- If the methods tried do not give the same result,reflect and\ntry again until you have two methods that have the same result.\n- If you still cannot arrive to a consistent result, say that\nyou are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful\nand thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,\nONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.\n- **ALWAYS**, as part of your "Final Answer", explain how you got\nto the answer on a section that starts with: "\n\nExplanation:\n".\nIn the explanat

In [28]:
question = "How may patients deceased in July 2020 from Texas"

agent.invoke(prompt_prefix + question + prompt_suffix)



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the state is Texas and the date is in July 2020. Then, I can sum the 'deathIncrease' column to get the total number of deaths in that period.
Action: python_repl_ast
Action Input: df[(df['state'] == 'TX') & (df['date'].dt.year == 2020) & (df['date'].dt.month == 7)]['deathIncrease'].sum()4145The sum of the 'deathIncrease' column for Texas in July 2020 is 4145. This means that 4145 patients deceased in July 2020 from Texas. However, to be sure of this result, I should try another method.
Action: python_repl_ast
Action Input: df.loc[(df['state'] == 'TX') & (df['date'].dt.year == 2020) & (df['date'].dt.month == 7), 'deathIncrease'].sum()4145The sum of the 'deathIncrease' column for Texas in July 2020 is also 4145 using the second method. This confirms that the result from the first method is correct.
Final Answer: 4145 patients deceased in Texas in July 2020.

Explanation:
I filtered 

{'input': '\nFirst set the pandas display options to show all the columns,\nget the column names, then answer the question.\nHow may patients deceased in July 2020 from Texas\n- **ALWAYS** before giving the Final Answer, try another method.\nThen reflect on the answers of the two methods you did and ask yourself\nif it answers correctly the original question.\nIf you are not sure, try another method.\n- If the methods tried do not give the same result,reflect and\ntry again until you have two methods that have the same result.\n- If you still cannot arrive to a consistent result, say that\nyou are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful\nand thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,\nONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.\n- **ALWAYS**, as part of your "Final Answer", explain how you got\nto the answer on a section that starts with: "\n\nExplanation:\n".\nIn the explanation, me

## Create a Gradio interface to ask questions

In [15]:
def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
#        history_langchain_format.append(AIMessage(content=ai))

    # Use the agent to generate the response
    response = agent.invoke(prompt_prefix + "Question: " + message + "Context: " + str(history_langchain_format) + prompt_suffix)

    return response['output']

# Launch the Gradio interface with the prediction function and subtitle
gr.ChatInterface(fn=predict, title="CSV Wizard", description="Answering questions from Covid statistics csv").launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9739a1995101ca934f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## SQL Agent - Connecting to SQL Database

In [16]:
from sqlalchemy import create_engine
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

### Create SQLite database

In [29]:
# Path to your SQLite database file
database_file_path = "./db/test.db"

# Create database if it doesn't exist
os.makedirs(os.path.dirname(database_file_path), exist_ok=True)

# Create an engine to connect to the SQLite database
engine = create_engine(f'sqlite:///{database_file_path}')

# create month_year column in dataframe
df['month_year'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m')

df.to_sql(
    'all_states_history',
    con=engine,
    if_exists='replace',
    index=False
)

20780

## Prepare SQL prompts

In [18]:
sql_prefix_prompt = """

You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct {dialect} query
to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most {top_k} results.
- You can order the results by a relevant column to return the most
interesting examples in the database.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it.If you get an error
while executing a query,rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.)
to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- Your response should be in Markdown. However, **when running  a SQL Query
in "Action Input", do not include the markdown backticks**.
Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- If the question does not seem related to the database, just return
"I don\'t know" as the answer.
- Only use the below tools. Only use the information returned by the
below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the
tools below.

## Tools:

"""

In [19]:
sql_format_instructions = """

## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input:
SELECT TOP (10) [death]
FROM covidtracking
WHERE state = 'TX' AND date LIKE '2020%'

Observation:
[(27437.0,), (27088.0,), (26762.0,), (26521.0,), (26472.0,), (26421.0,), (26408.0,)]
Thought:I now know the final answer
Final Answer: There were 27437 people who died of covid in Texas in 2020.

Explanation:
I queried the `covidtracking` table for the `death` column where the state
is 'TX' and the date starts with '2020'. The query returned a list of tuples
with the number of deaths for each day in 2020. To answer the question,
I took the sum of all the deaths in the list, which is 27437.
I used the following query

```sql
SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'"
```
===> End of Example

"""

## Build the SQL agent

In [20]:
db = SQLDatabase.from_uri(f'sqlite:///{database_file_path}')
model = ChatOpenAI(temperature=0.5, model='gpt-4')
toolkit = SQLDatabaseToolkit(db=db, llm=model)

agent_executor_SQL = create_sql_agent(
    prefix=sql_prefix_prompt,
    format_instructions = sql_format_instructions,
    llm=model,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

## Start asking questions

In [21]:
question = """
How may patients were hospitalized by year and month?
"""

agent_executor_SQL.invoke(question)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""all_states_historyThe `all_states_history` table seems relevant to the question. I should check its schema to understand its structure and the data it contains.
Action: sql_db_schema
Action Input: "all_states_history" 
CREATE TABLE all_states_history (
	date DATETIME, 
	state TEXT, 
	death FLOAT, 
	"deathConfirmed" FLOAT, 
	"deathIncrease" BIGINT, 
	"deathProbable" FLOAT, 
	hospitalized FLOAT, 
	"hospitalizedCumulative" FLOAT, 
	"hospitalizedCurrently" FLOAT, 
	"hospitalizedIncrease" BIGINT, 
	"inIcuCumulative" FLOAT, 
	"inIcuCurrently" FLOAT, 
	negative FLOAT, 
	"negativeIncrease" BIGINT, 
	"negativeTestsAntibody" FLOAT, 
	"negativeTestsPeopleAntibody" FLOAT, 
	"negativeTestsViral" FLOAT, 
	"onVentilatorCumulative" FLOAT, 
	"onVentilatorCurrently" FLOAT, 
	positive FLOAT, 
	"positiveCasesViral" FLOAT, 
	"positiveIncrease" BIGINT, 
	"positiveScore" BIGINT, 
	"positiveTestsAntibody" FLOAT, 
	"positiv

{'input': '\nHow may patients were hospitalized by year and month?\n',
 'output': 'The total number of patients hospitalized by year and month is as follows:\n\n- January 2020: 0\n- February 2020: 0\n- March 2020: 32,221\n- April 2020: 784,533\n- May 2020: 2,561,872\n- June 2020: 4,038,095\n- July 2020: 5,535,849\n- August 2020: 7,723,933\n- September 2020: 8,799,572\n- October 2020: 10,414,102\n- November 2020: 12,283,586\n- December 2020: 16,262,903\n- January 2021: 20,134,520\n- February 2021: 20,714,612\n- March 2021: 5,405,727\n\nExplanation: I queried the `all_states_history` table for the sum of the `hospitalized` column grouped by `year` and `month`. The query returned a list of tuples with the total number of people hospitalized for each month of each year. The query I used is:\n\n```sql\nSELECT year, month, SUM(hospitalized) as total_hospitalized FROM all_states_history GROUP BY year, month ORDER BY year, month\n```'}

In [22]:
question = """
how many deaths were from Texas by year and month?
"""

agent_executor_SQL.invoke(question)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""all_states_historyThe `all_states_history` table seems to be the most relevant one for this question. I should check its schema to see what columns it has.
Action: sql_db_schema
Action Input: all_states_history
CREATE TABLE all_states_history (
	date DATETIME, 
	state TEXT, 
	death FLOAT, 
	"deathConfirmed" FLOAT, 
	"deathIncrease" BIGINT, 
	"deathProbable" FLOAT, 
	hospitalized FLOAT, 
	"hospitalizedCumulative" FLOAT, 
	"hospitalizedCurrently" FLOAT, 
	"hospitalizedIncrease" BIGINT, 
	"inIcuCumulative" FLOAT, 
	"inIcuCurrently" FLOAT, 
	negative FLOAT, 
	"negativeIncrease" BIGINT, 
	"negativeTestsAntibody" FLOAT, 
	"negativeTestsPeopleAntibody" FLOAT, 
	"negativeTestsViral" FLOAT, 
	"onVentilatorCumulative" FLOAT, 
	"onVentilatorCurrently" FLOAT, 
	positive FLOAT, 
	"positiveCasesViral" FLOAT, 
	"positiveIncrease" BIGINT, 
	"positiveScore" BIGINT, 
	"positiveTestsAntibody" FLOAT, 
	"positiveTestsAn

{'input': '\nhow many deaths were from Texas by year and month?\n',
 'output': "The number of deaths in Texas by year and month are as follows:\n\n- March 2020: 227\n- April 2020: 11,438\n- May 2020: 39,119\n- June 2020: 61,207\n- July 2020: 120,655\n- August 2020: 304,998\n- September 2020: 431,275\n- October 2020: 523,085\n- November 2020: 590,591\n- December 2020: 758,658\n- January 2021: 984,700\n- February 2021: 1,123,403\n- March 2021: 306,654\n\nExplanation: I queried the `all_states_history` table for the `year`, `month`, and `SUM(death)` columns where the state is 'TX'. I grouped the results by `year` and `month` and ordered the results by `year` and `month`. The query returned a list of tuples with the year, month, and total number of deaths for each month in Texas. \n\nI used the following query:\n\n```sql\nSELECT year, month, SUM(death) as total_deaths\nFROM all_states_history\nWHERE state = 'TX'\nGROUP BY year, month\nORDER BY year, month\nLIMIT 30\n```"}

## Create a Gradio interface to ask questions

In [23]:
def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    # Use the agent to generate the response
    response = agent.invoke("Question: " + message + "Context: " + str(history_langchain_format))
    return response['output']

# Launch the Gradio interface with the prediction function and subtitle
gr.ChatInterface(fn=predict, title="SQL Wizard", description="Answering questions from Covid statistics Database").launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://485389dbdafcc737f7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
